<h1>PyTorch Neural Net to train Car</h1>

This notebook is the start of project 2, using the example of kruisjes-rondjes.ipynb form Ruud. In the second notebook we switch to the use of a DataLoader.

In [1]:
import pandas as pd
import torch
from torch import nn
from torch import optim

import matplotlib.pyplot as plt

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
# We move our tensor to the GPU if available

# if torch.cuda.is_available():
#    tensor = tensor.to("cuda")

In [3]:
# import data in pandas
df_train = pd.read_csv('../simpylc/sonar_1.samples', delim_whitespace=True,
                 header=None, index_col=False)

In [4]:
# check data type -> float, not object
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2508 entries, 0 to 2507
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       2508 non-null   float64
 1   1       2508 non-null   float64
 2   2       2508 non-null   float64
 3   3       2508 non-null   float64
dtypes: float64(4)
memory usage: 78.5 KB


In [5]:
# check seperator -> 4 columns present?
df_train.head()

,0,1,2,3
0,1.4079,1.2328,1.7440,-22.0
1,1.4082,1.2330,1.7441,-22.0
2,1.4062,1.2314,1.7429,-22.0
3,1.4062,1.2314,1.7429,-22.0
4,1.4011,1.2274,1.7392,-22.0


In [6]:
# make all Tensors of dtype float64 to avoid errors
torch.set_default_tensor_type(torch.DoubleTensor)

In [7]:
# make tensor from df
train_tensor = torch.tensor(df_train.values)

In [8]:
trainingSetX = train_tensor[:,0:3]

In [9]:
print(trainingSetX)

tensor([[1.4079, 1.2328, 1.7440],
        [1.4082, 1.2330, 1.7441],
        [1.4062, 1.2314, 1.7429],
        ...,
        [1.1076, 0.7148, 3.1654],
        [1.0626, 0.6836, 2.0476],
        [1.0626, 0.6836, 2.0476]])


In [10]:
trainingSetY = train_tensor[:,3]

In [11]:
print(trainingSetX.shape)
print(trainingSetY.shape)

torch.Size([2508, 3])
torch.Size([2508])


In [12]:
class NeuralNetwork(nn.Module):
    
    def __init__(self, inputs, outputs):
        
        super().__init__()
        
        # Simple linear connections
        self.inOut = nn.Linear(inputs, outputs)
        
        # Don't use Sigmoid as this outputs y = (0,1)
        # self.sigmoid = nn.Sigmoid()
        # Use tanh as this outputs y = (-1,1)
        self.tanh = nn.Tanh()
        # or even better: don't use activation, as this is a regression

    def forward(self, x):
        
        # return self.sigmoid(self.inOut(x))
        return self.tanh(self.inOut(x))
        # pass
        

In [13]:
print(x.shape)
print(y.shape)

NameError: name 'x' is not defined

In [17]:
#----------------------------------------

epochs = 1000

learningRate = 0.1

#----------------------------------------


model = NeuralNetwork(3,1)
# model = NeuralNetwork(3,1).to(device)

# print(model)

#----------------------------------------

losses = []

# Take MSE loss as loss function (one output in a regression model)
lossFunction = nn.MSELoss()

# Use stochastic gradient descent as optimizer, use weights and biases of model
# this is the connection between mdoel and gradinet descent
gradientDescent = torch.optim.SGD(model.parameters(), lr=learningRate)

# gradientDescent = torch.optim.RMSprop(model.parameters(), lr=learningRate)
# for better results?

#----------------------------------------


for i in range(epochs):
    # print(x.dtype)
    
    for x, y in zip(trainingSetX, trainingSetY):
        
        # Reset the gradient delta's (dw, db)
        gradientDescent.zero_grad()
        # x = x.to(device)
        # y = y.to(device)
        # print(x.get_device(), y.get_device())
        
        # Forward step
        yhat = model.forward(x)

        # Compute loss
        loss = lossFunction(yhat, y)
        
        # Keep track of loss
        losses.append(loss.item())

        # Apply gradient descent (via backpropagation)
        loss.backward()

        # Use w += -step * dw * learnRate to update the model
        gradientDescent.step()



In [ ]:
#----------------------------------------

fig, ax = plt.subplots()
ax.set(xlabel='Epoch', ylabel='Cost', title="Training Cost")

plt.plot([x for x in range(4 * epochs)], losses, 'red')
plt.show()

#----------------------------------------